<a href="https://colab.research.google.com/github/lorifin/PAL/blob/main/pred_ventes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install notion-client slack_sdk openai pandas matplotlib requests

import pandas as pd
import openai
from datetime import datetime
import matplotlib.pyplot as plt
import requests
from notion_client import Client
from slack_sdk import WebClient
import os
print(os.listdir())

**⚠️ IMPORTANT : Ce notebook ne contient pas de clé privée. Ajoutez vos clés d’API (Slack, Notion, Jira, OpenAI) dans un fichier `.env` ou une cellule dédiée non partagée.**

# --- PARAMÈTRES ---
OPENAI_API_KEY = ""                # Clé OpenAI
NOTION_TOKEN = ""              # Token Notion API
NOTION_DATABASE_ID = ""  # ID de ta base Notion
SLACK_BOT_TOKEN = ""             # Bot User OAuth Token Slack
SLACK_CHANNEL = ""               # Ou ID du canal Slack

N_MOIS_PRED = 3  # Nombre de mois à prédire

# --- (1) Exemple données fictives ---
import numpy as np
mois = pd.date_range("2023-01-01", periods=18, freq="M")
ventes = np.random.randint(2000, 3000, size=len(mois))
df = pd.DataFrame({"Date": mois, "Montant": ventes})
df["Mois"] = df["Date"].dt.strftime('%B %Y')

# --- (2) Formatage pour GPT ---
historique_txt = ""
for i, row in df.iterrows():
    historique_txt += f"{row['Date'].strftime('%B %Y')}: {row['Montant']}€\n"

prompt = f"""
Voici l’historique des ventes mensuelles (en €) :
{historique_txt}

1) Estime-moi les ventes pour les {N_MOIS_PRED} prochains mois (donne le montant par mois).
2) Résume la tendance (hausse/baisse/saisonnalité) en 2 phrases.
3) Donne-moi 2 conseils pour augmenter les ventes ou anticiper une baisse.
"""

openai.api_key = OPENAI_API_KEY
response = openai.chat.completions.create(
    model="gpt-4o",  # ou "gpt-3.5-turbo"
    messages=[{"role": "user", "content": prompt}],
    max_tokens=300,
    temperature=0.2,
)

resultat_gpt = response.choices[0].message.content

print("---- PRÉDICTION GPT ----")
print(resultat_gpt)

# --- (3) Envoi vers Notion ---
notion = Client(auth=NOTION_TOKEN)
new_page = notion.pages.create(
    parent={"database_id": NOTION_DATABASE_ID},
    properties={
        "Nom": {"title": [{"text": {"content": f"Prédiction du {datetime.now().strftime('%d/%m/%Y')}"} }]},
        "Synthèse": {"rich_text": [{"text": {"content": resultat_gpt}}]},
        "Date": {"date": {"start": datetime.now().strftime('%Y-%m-%d')}},
        # Ajoute ici les propriétés selon le schéma de ta base Notion (adapte les noms!)
    }
)
print("✅ Prédiction enregistrée dans Notion !")

# --- (4) Envoi sur Slack ---
slack_client = WebClient(token=SLACK_BOT_TOKEN)
slack_message = f"*Prédiction commerciale automatique* :\n```{resultat_gpt}```"
try:
    slack_client.chat_postMessage(channel=SLACK_CHANNEL, text=slack_message)
    print("✅ Prédiction envoyée sur Slack !")
except Exception as e:
   print(f"❌ Erreur Slack : {e}")

# --- (5) (Optionnel) Graphe rapide
plt.plot(df['Date'], df['Montant'], marker='o', label='Historique')
plt.title('Ventes mensuelles')
plt.xlabel('Mois')
plt.ylabel('Montant (€)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.legend()
plt.show()

import matplotlib.pyplot as plt

plt.plot(df['Date'], df['Montant'], marker='o', label='Historique')
plt.title('Ventes mensuelles')
plt.xlabel('Mois')
plt.ylabel('Montant (€)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.legend()
plt.savefig("ventes_graph.png")  # Sauvegarde le graphe
plt.close()

from slack_sdk import WebClient

slack_client = WebClient(token=SLACK_BOT_TOKEN)
channel_id = ""  # ou #nom_du_canal

import requests
import base64
import os

import requests
import base64
import os

def upload_to_github_and_share(token, file_path, message="Graphique des prédictions"):
    """
    Utilise l'image déjà pushée sur GitHub et partage l'URL raw
    """
    channel_id = ""

    try:
        # L'image est déjà sur GitHub après le push
        github_raw_url = f""

        # Partager sur Slack avec l'URL GitHub
        slack_headers = {"Authorization": f"Bearer {token}"}
        slack_payload = {
            "channel": channel_id,
            "blocks": [
                {
                    "type": "section",
                    "text": {
                        "type": "mrkdwn",
                        "text": f"*{message}*"
                    }
                },
                {
                    "type": "image",
                    "image_url": github_raw_url,
                    "alt_text": "Graphique des prédictions"
                }
            ]
        }

        slack_response = requests.post(
            "https://slack.com/api/chat.postMessage",
            headers=slack_headers,
            json=slack_payload
        )

        if slack_response.json().get('ok'):
            print("✅ Graphique partagé avec succès via GitHub!")
            return True
        else:
            print(f"❌ Erreur Slack: {slack_response.json().get('error')}")
            return False

    except Exception as e:
        print(f"❌ Erreur GitHub: {e}")
        return False

def send_as_data_uri(token, file_path, message="Graphique des prédictions"):
    """
    Envoie l'image comme data URI directement dans le message
    """
    channel_id = ""

    try:
        with open(file_path, 'rb') as f:
            file_data = f.read()
            file_b64 = base64.b64encode(file_data).decode('utf-8')

        # Créer un data URI
        data_uri = f"data:image/png;base64,{file_b64}"

        slack_headers = {"Authorization": f"Bearer {token}"}

        # Essayer avec blocks
        slack_payload = {
            "channel": channel_id,
            "text": message,
            "blocks": [
                {
                    "type": "section",
                    "text": {
                        "type": "mrkdwn",
                        "text": f"*{message}*"
                    }
                },
                {
                    "type": "image",
                    "image_url": data_uri,
                    "alt_text": "Graphique"
                }
            ]
        }

        slack_response = requests.post(
            "https://slack.com/api/chat.postMessage",
            headers=slack_headers,
            json=slack_payload
        )

        if slack_response.json().get('ok'):
            print("✅ Graphique envoyé comme data URI!")
            return True
        else:
            print(f"❌ Erreur data URI: {slack_response.json().get('error')}")
            return False

    except Exception as e:
        print(f"❌ Erreur data URI: {e}")
        return False

def send_image_as_snippet(token, file_path, message="Graphique"):
    """
    Alternative: Envoyer l'image comme snippet de code
    """
    channel_id = ""

    try:
        with open(file_path, 'rb') as f:
            file_data = f.read()
            file_b64 = base64.b64encode(file_data).decode('utf-8')

        headers = {"Authorization": f"Bearer {token}"}

        # Utiliser files.upload v2 si disponible
        payload = {
            "channel": channel_id,
            "content": file_b64,
            "filename": "graph.png",
            "filetype": "png",
            "initial_comment": message,
            "title": "Graphique Prédictions"
        }

        response = requests.post(
            "https://slack.com/api/files.upload",
            headers=headers,
            data=payload
        )

        result = response.json()
        if result.get('ok'):
            print("✅ Snippet envoyé avec succès!")
            return True
        else:
            print(f"❌ Erreur snippet: {result.get('error')}")
            return False

    except Exception as e:
        print(f"❌ Erreur snippet: {e}")
        return False

def create_slack_webhook_solution(webhook_url, file_path, message="Graphique"):
    """
    Solution webhook - Vous devez créer un webhook dans votre app Slack
    """
    import json

    try:
        with open(file_path, 'rb') as f:
            file_data = f.read()
            file_b64 = base64.b64encode(file_data).decode('utf-8')

        # Payload pour webhook
        payload = {
            "text": message,
            "blocks": [
                {
                    "type": "section",
                    "text": {
                        "type": "mrkdwn",
                        "text": f"*{message}*"
                    }
                },
                {
                    "type": "image",
                    "image_url": f"data:image/png;base64,{file_b64}",
                    "alt_text": "Graphique"
                }
            ]
        }

        response = requests.post(webhook_url, json=payload)
        return response.status_code == 200

    except Exception as e:
        print(f"❌ Erreur webhook: {e}")
        return False

# Fonction principale
def send_graph_to_slack(token, file_path, message="Graphique des prédictions"):
    """
    Fonction principale qui essaie plusieurs méthodes
    """
    print("🔄 Tentative d'envoi du graphique...")

    # Vérifier que le fichier existe
    if not os.path.exists(file_path):
        print(f"❌ Fichier {file_path} non trouvé")
        print("Fichiers disponibles:", [f for f in os.listdir('.') if f.endswith(('.png', '.jpg', '.jpeg'))])
        return False

    # Méthode 1: Via GitHub (l'image est déjà pushée)
    print("📤 Méthode 1: Via GitHub...")
    if upload_to_github_and_share(token, file_path, message):
        return True

    # Méthode 2: Data URI
    print("📤 Méthode 2: Data URI...")
    if send_as_data_uri(token, file_path, message):
        return True

    print("❌ Toutes les méthodes ont échoué")
    return False

# Utilisation
if __name__ == "__main__":
    slack_token = ""  # Remplacez par votre token
    graph_file = "ventes_graph.png"

    send_graph_to_slack(slack_token, graph_file)

# Utilisation
slack_token = ""  # Remplacez par votre token
graph_file = "ventes_graph.png"  # Nom de votre fichier graphique

# Vérifiez que le fichier existe
if os.path.exists(graph_file):
    send_graph_to_slack(slack_token, graph_file)
else:
    print(f"❌ Fichier {graph_file} non trouvé")
    print("Fichiers disponibles:", [f for f in os.listdir('.') if f.endswith('.png')])



